In [31]:
import pandas as pd

# df = pd.read_csv('/opt/home/revoli/eva/tSBDD/result/normal_affinity/core_timetest_round0_ensemble.csv')  # with_out_negatives
df = pd.read_csv('/opt/home/revoli/eva/tSBDD/result/core_timetest_round0_ensemble.csv')  # Interformer
no_recp_overlap = [x.strip() for x in open('/opt/home/revoli/data_worker/interformer/train/diffdock_splits/timesplit_test_no_rec_overlap').readlines()] # read no_recp_overlap
#
df['pIC50'] = df['pIC50'].map(lambda x: abs(x))
#
df = df[df['task'] != 'core']  # [Docking] exclude coreset
# df = df[df['Target'].isin(no_recp_overlap)]# [Docking] only no_rec_overlap
#
# df = df[df['task'] == 'core'] # [Affinity] only including coreset
#
df

,Target,pose_rank,energy,rmsd,Year,Uniprot,ProteinName,Resolution,pIC50,Unit,Molecule ID,task,cid,pred_pIC50,pred_pose
0,6v5l,0,-441.150434,4.48381,2018,P01116,GTPASE KRAS,NMR,3.65,Kd=224.17uM,QPD,test,0,2.817627,0.000937
1,6v5l,1,-424.170435,2.81796,2018,P01116,GTPASE KRAS,NMR,3.65,Kd=224.17uM,QPD,test,0,3.661621,0.008685
2,6v5l,2,-415.401541,6.51086,2018,P01116,GTPASE KRAS,NMR,3.65,Kd=224.17uM,QPD,test,0,2.140625,0.003050
3,6v5l,3,-414.926505,6.60153,2018,P01116,GTPASE KRAS,NMR,3.65,Kd=224.17uM,QPD,test,0,2.936035,0.003519
4,6v5l,4,-412.334288,1.60472,2018,P01116,GTPASE KRAS,NMR,3.65,Kd=224.17uM,QPD,test,0,4.870117,0.138464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,6uwv,16,-614.803612,4.02473,2019,P56817,BETA-SECRETASE 1,1.47,8.00,IC50=0.01uM,QK7,test,0,5.343750,0.006770
7178,6uwv,17,-613.444484,8.27286,2019,P56817,BETA-SECRETASE 1,1.47,8.00,IC50=0.01uM,QK7,test,0,5.075684,0.006472
7179,6uwv,18,-608.233862,3.46001,2019,P56817,BETA-SECRETASE 1,1.47,8.00,IC50=0.01uM,QK7,test,0,7.200195,0.097255
7180,6uwv,19,-607.252606,7.78167,2019,P56817,BETA-SECRETASE 1,1.47,8.00,IC50=0.01uM,QK7,test,0,4.294922,0.000927


In [32]:
dock_ac_df = df.copy()
dock_ac_df = dock_ac_df[dock_ac_df['pose_rank'] != 20]  # exclude crystal
def eval(df, name, top=1):
  name = name + f'-{top}'
  top1 = df.groupby('Target').head(top)
  top1 = top1.groupby('Target').min()
  print(f"N:{len(top1)}")
  print(name, (top1['rmsd'] < 2.).sum() / len(top1))
  print(name + '_median', top1['rmsd'].median())

# PoseScore
pred_df = dock_ac_df.sort_values('pred_pose', ascending=0)
eval(pred_df, "PoseScore")
eval(pred_df, "PoseScore", top=5)
print('=' * 100)
# Energy
pred_df = dock_ac_df.sort_values('pose_rank', ascending=1)
eval(pred_df, "EnergyRank")
eval(pred_df, "EnergyRank", top=5)

N:342
PoseScore-1 0.5497076023391813
PoseScore-1_median 1.827325
N:342
PoseScore-5 0.7105263157894737
PoseScore-5_median 1.40237
N:342
EnergyRank-1 0.43859649122807015
EnergyRank-1_median 2.493175
N:342
EnergyRank-5 0.6257309941520468
EnergyRank-5_median 1.481785


In [37]:
# Coreset's affinity
# pred_pose, pose_rank
no_crystal_df = df[df['pose_rank'] != 20]
crystal = df[df['pose_rank'] == 20]  # using crystal to calculate pIC50

def eval_R(eval_df, name):
  print(name)
  key = 'pred_pIC50'  # energy, pred_pIC50
  print("Pearson", eval_df.corr(numeric_only=True)['pIC50'][key])
  print("Spearman", eval_df.corr('spearman', numeric_only=True)['pIC50'][key])

# rank by PoseScore
top1 = no_crystal_df.sort_values('pred_pose', ascending=0).groupby('Target').head(1)
eval_R(top1, 'pred_pose')
# rank by Energy
pose_1 = no_crystal_df.sort_values('pose_rank', ascending=1).groupby('Target').head(1) 
eval_R(pose_1, 'pose_rank')
# Crystal
eval_R(crystal, 'pred_pIC50')
###
top1

pred_pose
Pearson 0.26071453886087237
Spearman 0.49804801174076846
pose_rank
Pearson 0.154337580002039
Spearman 0.33284354275573536
pred_pIC50
Pearson 0.6811693471083308
Spearman 0.6456834159311967


,Target,pose_rank,energy,rmsd,Year,Uniprot,ProteinName,Resolution,pIC50,Unit,Molecule ID,task,cid,pred_pIC50,pred_pose
6450,6qw8,3,-742.939357,0.745528,2019,G3G192,BETA-LACTAMASE,1.10,6.40,IC50=400nM,MK7,test,0,7.225586,9.993193e-01
1809,6hhr,3,-869.557854,0.322045,2019,P07900,HEAT SHOCK PROTEIN HSP90-ALPHA,2.00,7.01,Kd=98nM,G5E,test,0,7.334961,9.992741e-01
948,6fe5,3,-916.640026,0.715967,2019,Q04609,GLUTAMATE CARBOXYPEPTIDASE 2,1.52,9.30,IC50=0.50nM,D6E,test,0,7.638672,9.992712e-01
1249,6g3c,10,-842.776389,0.758254,2019,Q506Q0,TYROSINE-PROTEIN KINASE,1.60,6.91,IC50=123nM,EKT,test,0,7.950195,9.991065e-01
3087,6jsf,0,-1148.397730,0.456386,2019,P56817,BETA-SECRETASE 1,2.30,8.48,IC50=3.3nM,C7X,test,0,7.972656,9.991035e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,6d08,4,-1124.670871,6.205300,2019,P83916,CHROMOBOX PROTEIN HOMOLOG 1,2.10,5.29,Kd=5.1uM,16-mer,test,0,-12.179688,1.516650e-06
3604,6mji,13,-1138.392412,10.936700,2019,K7N5M3,"T CELL RECEPTOR ALPHA VARIABLE 11,T CELL RECEP...",2.30,7.29,Kd=51.5nM,JTD,test,0,-11.219727,1.464963e-06
4000,6n97,10,-1104.739002,10.484500,2019,P52045,METHYLMALONYL-COA DECARBOXYLASE,1.75,5.15,Ki=7.1uM,KGP,test,0,-13.270508,1.382792e-06
6622,6rpg,7,-1269.270579,7.858590,2019,Q9NZQ7,PROGRAMMED CELL DEATH 1 LIGAND 1,2.70,8.52,IC50=3.0nM,KDW,test,0,-7.422104,1.054024e-06


In [21]:
# RMSD V.S []
df.corr(numeric_only=True)

,pose_rank,energy,rmsd,Year,pIC50,cid,pred_pIC50,pred_pose
pose_rank,1.000000e+00,0.144455,0.255382,-1.183773e-14,-8.610367e-17,NaN,-0.084964,-0.231827
energy,1.444550e-01,1.000000,-0.142425,-3.766874e-02,-5.180000e-01,NaN,0.197960,0.066009
rmsd,2.553822e-01,-0.142425,1.000000,1.064175e-02,4.678375e-02,NaN,-0.535717,-0.614517
Year,-1.183773e-14,-0.037669,0.010642,1.000000e+00,8.357196e-02,NaN,-0.002019,0.034437
pIC50,-8.610367e-17,-0.518000,0.046784,8.357196e-02,1.000000e+00,NaN,0.151662,-0.012428
cid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pred_pIC50,-8.496424e-02,0.197960,-0.535717,-2.019113e-03,1.516624e-01,NaN,1.000000,0.451277
pred_pose,-2.318267e-01,0.066009,-0.614517,3.443662e-02,-1.242814e-02,NaN,0.451277,1.000000
